In [1]:
from sklearn.model_selection import KFold
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import datasets, transforms
import timm
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from pathlib import Path

In [2]:
epochs = 5
lr = 0.0001
device = 'cuda'
k_folds = 3

dataset_dir = Path('C:/Users/takatsuto/MLBook/data/Mushrooms')

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
dataset = datasets.ImageFolder(dataset_dir, transform)

model = timm.create_model('swin_base_patch4_window7_224.ms_in22k', pretrained=True, num_classes=2)
#model = timm.create_model('resnet50', pretrained=True, num_classes=2)
model = model.to(device)

# 損失関数と最適化手法
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

kf = KFold(n_splits=k_folds, shuffle=True)

# 結果保存用のリスト
train_acc_list = []
val_acc_list = []
train_loss_list = []
val_loss_list = []

In [5]:
for fold, (train_idx, val_idx) in enumerate(kf.split(np.arange(len(dataset)))):    
    print(f"Fold {fold+1}/{k_folds}")

    # トレーニングと検証データの分割
    train_subset = Subset(dataset, train_idx)
    val_subset = Subset(dataset, val_idx)

    train_loader = DataLoader(dataset=train_subset, batch_size=16, shuffle=True)
    val_loader = DataLoader(dataset=val_subset, batch_size=16, shuffle=False)

    # モデルの再初期化
    model = model.to(device)
    
    best_loss = None
    # エポックループ
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        epoch_accuracy = 0

        # トレーニングループ
        for data, label in tqdm(train_loader):
            data = data.to(device)
            label = label.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, label)

            loss.backward()
            optimizer.step()

            # 精度の計算
            acc = (output.argmax(dim=1) == label).float().mean()
            epoch_accuracy += acc / len(train_loader)
            epoch_loss += loss / len(train_loader)

        # 検証ループ
        model.eval()
        epoch_val_loss = 0
        epoch_val_accuracy = 0
        with torch.no_grad():
            for data, label in val_loader:
                data = data.to(device)
                label = label.to(device)

                output = model(data)
                loss = criterion(output, label)

                acc = (output.argmax(dim=1) == label).float().mean()
                epoch_val_accuracy += acc / len(val_loader)
                epoch_val_loss += loss / len(val_loader)

        print(f"Epoch {epoch+1}/{epochs} - loss: {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss: {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}")

        train_acc_list.append(epoch_accuracy.item())
        val_acc_list.append(epoch_val_accuracy.item())
        train_loss_list.append(epoch_loss.item())
        val_loss_list.append(epoch_val_loss.item())

        # 最良モデルの保存
        if best_loss is None or epoch_val_loss < best_loss:
            best_loss = epoch_val_loss
            model_path = f'model_fold{fold+1}_best.pth'
            torch.save(model.state_dict(), model_path)

    print(f"Fold {fold+1} training completed.\n")

Fold 1/3


  0%|          | 0/74 [00:00<?, ?it/s]

Epoch 1/5 - loss: 0.4009 - acc: 0.8319 - val_loss: 0.2029 - val_acc: 0.9223


  0%|          | 0/74 [00:00<?, ?it/s]

Epoch 2/5 - loss: 0.1370 - acc: 0.9510 - val_loss: 0.2781 - val_acc: 0.8834


...

Epoch 3/5 - loss: 0.0393 - acc: 0.9882 - val_loss: 0.0263 - val_acc: 0.9899


  0%|          | 0/74 [00:00<?, ?it/s]

Epoch 4/5 - loss: 0.0197 - acc: 0.9958 - val_loss: 0.0236 - val_acc: 0.9899


  0%|          | 0/74 [00:00<?, ?it/s]

Epoch 5/5 - loss: 0.0068 - acc: 0.9975 - val_loss: 0.0380 - val_acc: 0.9882
Fold 3 training completed.

